In [ ]:
import pandas as pd

def preparar_base_final(archivo_2016, archivo_2023, salida):
    # =========================
    # 1. COLUMNAS NECESARIAS
    # =========================
    columnas = [
        'Pais', 'Region', 'PIB per capita', 'Poblacion (millones)',
        'Huella ecologica total (consumo)', 'Biocapacidad total',
        'Deficit de biocapacidad', 'Tierras requeridas'
    ]

    # =========================
    # 2. CARGAR ARCHIVOS
    # =========================
    df16 = pd.read_excel(archivo_2016)
    df23 = pd.read_excel(archivo_2023)

    # =========================
    # 3. NORMALIZAR PAÍSES
    # =========================
    for df in (df16, df23):
        df['Pais'] = df['Pais'].astype(str).str.strip().str.lower()

    # =========================
    # 4. SELECCIONAR COLUMNAS Y LIMPIAR
    # =========================
    df16 = df16[columnas].dropna()
    df23 = df23[columnas].dropna()

    # =========================
    # 5. MANTENER PAÍSES COMUNES
    # =========================
    paises_comunes = set(df16['Pais']).intersection(df23['Pais'])
    df16 = df16[df16['Pais'].isin(paises_comunes)]
    df23 = df23[df23['Pais'].isin(paises_comunes)]

    # =========================
    # 6. AGREGAR AÑO
    # =========================
    df16['Año'] = 2016
    df23['Año'] = 2023

    # =========================
    # 7. CONCATENAR
    # =========================
    df = pd.concat([df16, df23], ignore_index=True)

    # =========================
    # 8. LIMPIAR PIB PER CÁPITA
    # =========================
    df['PIB per capita'] = (
        df['PIB per capita']
        .astype(str)
        .str.replace(r'[^\d.]', '', regex=True)
    )
    df['PIB per capita'] = pd.to_numeric(df['PIB per capita'], errors='coerce')
    df = df.dropna(subset=['PIB per capita'])

    # =========================
    # 9. MAPEAR REGIÓN → CONTINENTE
    # =========================
    mapa_continentes = {
        'Northern/Eastern Europe': 'Europa',
        'European Union': 'Europa',
        'EU-27': 'Europa',
        'Other Europe': 'Europa',

        'Latin America': 'América',
        'North America': 'América',
        'Central America/Caribbean': 'América',
        'South America': 'América',

        'Middle East/Central Asia': 'Asia',
        'Asia-Pacific': 'Asia',

        'Africa': 'África',
        'Sub-Saharan Africa': 'África',

        'Oceania': 'Oceanía'
    }

    df['Continente'] = df['Region'].map(mapa_continentes)

    # =========================
    # 10. CLASIFICACIÓN BANCO MUNDIAL
    # =========================
    def clasificar_ingreso_bm(pib, año):
        if año == 2016:
            if pib <= 1025:
                return 'Ingreso bajo'
            elif pib <= 12475:
                return 'Ingreso medio'
            else:
                return 'Ingreso alto'
        elif año == 2023:
            if pib <= 1135:
                return 'Ingreso bajo'
            elif pib <= 13845:
                return 'Ingreso medio'
            else:
                return 'Ingreso alto'

    df['Nivel de ingreso'] = df.apply(
        lambda f: clasificar_ingreso_bm(
            f['PIB per capita'],
            f['Año']
        ),
        axis=1
    )

    # =========================
    # 11. GUARDAR EXCEL FINAL
    # =========================
    df.to_excel(salida, index=False)
    print("✔ Base final creada:", salida)

    return df


# =========================
# 12. EJECUCIÓN
# =========================
df_final = preparar_base_final(
    archivo_2016=r"C:/Users/HP/Documents/Jupiter/articulo, huella ecologica/huella ecologica 2016.xlsx",
    archivo_2023=r"C:/Users/HP/Documents/Jupiter/articulo, huella ecologica/Huella ecológica 2023.xlsx",
    salida=r"C:/Users/HP/Desktop/Jupiter/Base_Final_Huella_Ecologica_3.xlsx"
)

# =========================
# 13. VERIFICACIONES
# =========================
print("\nPaíses únicos:", df_final['Pais'].nunique())
print("Filas por año:")
print(df_final['Año'].value_counts())
print("\nDistribución por nivel de ingreso:")
print(df_final['Nivel de ingreso'].value_counts())